### ИНТЕРАКТИВНЫЙ ДАШБОРД

In [ ]:
# импорт необходимых библиотек
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np

In [ ]:
# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

# Нормализация категорий
df["Education"] = df["Education"].replace({"2n Cycle": "Pre-Graduate", "Basic": "Pre-Graduate"})
df["Marital_Status"] = df["Marital_Status"].replace({
    "Married": "Married/Together", "Together": "Married/Together",
    "Single": "Single", "Divorced": "Other", "Widow": "Other",
    "Alone": "Other", "Absurd": "Other", "YOLO": "Other"
})

# Подготовка данных
df['Age'] = 2024 - df['Year_Birth']  
df['Total_Children'] = df['Kidhome'] + df['Teenhome']  
df['Total_Spending'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 
                           'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1) 

# Очистка Income
df['Income'] = df['Income'].fillna(df['Income'].median())
df = df[df['Income'] < 600000]

# Создаем признак "Есть дети" для фильтра
df['Has_Children'] = (df['Total_Children'] > 0).astype(str)
df['Has_Children'] = df['Has_Children'].replace({'True': 'Есть дети', 'False': 'Нет детей'})

In [ ]:
# 3 ФИЛЬТРА
# Фильтр 1: Образование 
education_filter = widgets.SelectMultiple(
    options=sorted(df['Education'].unique()),
    value=list(df['Education'].unique()),
    description='Образование:',
    disabled=False,
    layout=widgets.Layout(width='350px', height='100px')
)
# Фильтр 2: Семейное положение 
marital_filter = widgets.SelectMultiple(
    options=sorted(df['Marital_Status'].unique()),
    value=list(df['Marital_Status'].unique()),
    description='Семья:',
    disabled=False,
    layout=widgets.Layout(width='350px', height='100px')
)

# Фильтр 3: Наличие детей
children_filter = widgets.RadioButtons(
    options=['Все', 'Есть дети', 'Нет детей'],
    value='Все',
    description='Дети:',
    disabled=False,
    layout=widgets.Layout(width='200px')
)

# Кнопка обновления
update_button = widgets.Button(
    description='Обновить дашборд',
    button_style='success',
    icon='check',
    layout=widgets.Layout(width='200px', height='40px')
)

# Вывод для дашборда
output = widgets.Output()

In [ ]:
# ФУНКЦИЯ ОБНОВЛЕНИЯ ДАШБОРДА
def update_dashboard(btn):
    with output:
        clear_output(wait=True)
        
        # Применяем фильтры
        filtered_df = df[
            (df['Education'].isin(education_filter.value)) &
            (df['Marital_Status'].isin(marital_filter.value))
        ]
        
        # Фильтр по детям
        if children_filter.value == 'Есть дети':
            filtered_df = filtered_df[filtered_df['Has_Children'] == 'Есть дети']
        elif children_filter.value == 'Нет детей':
            filtered_df = filtered_df[filtered_df['Has_Children'] == 'Нет детей']
        
        # Проверка на пустые данные
        if len(filtered_df) == 0:
            display(HTML("<h3 style='color: red;'>Нет данных с выбранными фильтрами</h3>"))
            return
        
        # 1. KPI КАРТОЧКИ (Средний доход и Средний чек)
        avg_income = filtered_df['Income'].mean()
        avg_spending = filtered_df['Total_Spending'].mean()
        total_customers = len(filtered_df)
        response_rate = filtered_df['Response'].mean() * 100
        
        # Создаем HTML для KPI
        kpi_html = f"""
        <div style='display: flex; justify-content: space-around; background-color: #f0f2f6; padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
            <div style='text-align: center;'>
                <h3 style='color: #1f77b4; margin: 0;'>{total_customers:,}</h3>
                <p style='color: gray; margin: 0;'>Клиентов</p>
            </div>
            <div style='text-align: center;'>
                <h3 style='color: #2ca02c; margin: 0;'>${avg_income:,.0f}</h3>
                <p style='color: gray; margin: 0;'>Средний доход</p>
            </div>
            <div style='text-align: center;'>
                <h3 style='color: #ff7f0e; margin: 0;'>${avg_spending:,.0f}</h3>
                <p style='color: gray; margin: 0;'>Средний чек</p>
            </div>
            <div style='text-align: center;'>
                <h3 style='color: #d62728; margin: 0;'>{response_rate:.1f}%</h3>
                <p style='color: gray; margin: 0;'>Отклик</p>
            </div>
        </div>
        """
        display(HTML(kpi_html))
        
        # 2. КОЛЬЦЕВАЯ ДИАГРАММА (Состав корзины)
        expense_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
                       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
        expense_names = ['Вино', 'Фрукты', 'Мясо', 'Рыба', 'Сладости', 'Золото']
        expense_data = filtered_df[expense_cols].mean()
        
        fig1 = go.Figure(data=[go.Pie(
            labels=expense_names,
            values=expense_data.values,
            hole=0.4,
            marker_colors=px.colors.qualitative.Set3,
            textinfo='percent+label',
            hoverinfo='label+percent+value',
            textposition='auto'
        )])
        fig1.update_layout(
            title='Состав корзины (средние траты по категориям)',
            height=400,
            showlegend=False,
            annotations=[dict(text=f'${avg_spending:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)]
        )
        
        # 3. ТОЧЕЧНАЯ ДИАГРАММА (Доход vs Траты) - КЛАСТЕРИЗАЦИЯ
        fig2 = go.Figure()
        
        # Раскрашиваем по отклику для кластеризации
        colors = ['#FF6B6B' if x == 0 else '#4ECDC4' for x in filtered_df['Response']]
        
        fig2.add_trace(go.Scatter(
            x=filtered_df['Income'],
            y=filtered_df['Total_Spending'],
            mode='markers',
            marker=dict(
                color=colors,
                size=8,
                opacity=0.6,
                line=dict(width=1, color='black')
            ),
            text=filtered_df.apply(lambda row: f"Возраст: {row['Age']}<br>Дети: {row['Has_Children']}<br>Образование: {row['Education']}", axis=1),
            hovertemplate='<b>Доход:</b> $%{x:,.0f}<br><b>Траты:</b> $%{y:,.0f}<br>%{text}<extra></extra>'
        ))
        
        # Добавляем линии средних
        fig2.add_hline(y=avg_spending, line_dash="dash", line_color="gray", 
                      annotation_text=f"Ср. траты: ${avg_spending:,.0f}", annotation_position="bottom right")
        fig2.add_vline(x=avg_income, line_dash="dash", line_color="gray",
                      annotation_text=f"Ср. доход: ${avg_income:,.0f}", annotation_position="top left")
        
        fig2.update_layout(
            title='Доход vs Траты (кластеризация клиентов)',
            xaxis_title='Доход ($)',
            yaxis_title='Общие траты ($)',
            height=400,
            hovermode='closest'
        )
        
        # 4. СТОЛБЧАТАЯ ДИАГРАММА (Каналы продаж)
        purchase_cols = ['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases']
        purchase_names = ['Веб-покупки', 'Каталог', 'Магазин', 'Со скидкой']
        purchase_data = filtered_df[purchase_cols].mean()
        
        fig3 = go.Figure(data=[
            go.Bar(
                name='Среднее количество',
                x=purchase_names,
                y=purchase_data.values,
                marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
                text=purchase_data.values.round(1),
                textposition='outside'
            )
        ])
        fig3.update_layout(
            title='Среднее количество покупок по каналам',
            xaxis_title='Канал продаж',
            yaxis_title='Среднее количество',
            height=350,
            showlegend=False
        )
        
        # 5. ГРУППИРОВАННАЯ ГИСТОГРАММА (Реакция на акции)
        cmp_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
        cmp_names = ['Кампания 1', 'Кампания 2', 'Кампания 3', 'Кампания 4', 'Кампания 5']
        
        # Считаем количество принявших каждую кампанию
        cmp_data = []
        for col in cmp_cols:
            cmp_data.append(filtered_df[col].sum())
        
        fig4 = go.Figure(data=[
            go.Bar(
                name='Приняли участие',
                x=cmp_names,
                y=cmp_data,
                marker_color='#9467bd',
                text=cmp_data,
                textposition='outside'
            )
        ])
        fig4.update_layout(
            title='Участие в маркетинговых кампаниях',
            xaxis_title='Кампания',
            yaxis_title='Количество клиентов',
            height=350,
            showlegend=False
        )
        
        # Отображаем графики в сетке
        # Первый ряд: кольцевая диаграмма и scatter plot
        col1, col2 = widgets.VBox([widgets.HTML("<h3>Состав корзины</h3>"), go.FigureWidget(fig1)]), \
                     widgets.VBox([widgets.HTML("<h3>Кластеризация</h3>"), go.FigureWidget(fig2)])
        display(widgets.HBox([col1, col2]))
        
        # Второй ряд: каналы продаж и участие в кампаниях
        col3, col4 = widgets.VBox([widgets.HTML("<h3>Каналы продаж</h3>"), go.FigureWidget(fig3)]), \
                     widgets.VBox([widgets.HTML("<h3>Участие в кампаниях</h3>"), go.FigureWidget(fig4)])
        display(widgets.HBox([col3, col4]))
        
        # Статистика по сегменту
        display(HTML(f"""
        <div style='background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin-top: 20px;'>
            <h4>Анализ выбранного сегмента:</h4>
            <p>• <b>Самый популярный товар:</b> {expense_names[np.argmax(expense_data.values)]} (${max(expense_data.values):,.0f})</p>
            <p>• <b>Любимый канал:</b> {purchase_names[np.argmax(purchase_data.values)]} ({max(purchase_data.values):.1f} покупок)</p>
            <p>• <b>Лучшая кампания:</b> {cmp_names[np.argmax(cmp_data)]} ({max(cmp_data)} откликов)</p>
            <p>• <b>Средний возраст:</b> {filtered_df['Age'].mean():.0f} лет</p>
        </div>
        """))

# Привязываем функцию к кнопке
update_button.on_click(update_dashboard)


In [ ]:
# ОТОБРАЖАЕМ ДАШБОРД
display(HTML("""
<h1 style='text-align: center; color: #2c3e50;'> Маркетинговый дашборд</h1>
<p style='text-align: center; color: #7f8c8d;'>Анализ клиентских сегментов и покупательского поведения</p>
<hr style='border: 1px solid #3498db;'>
"""))

# Отображаем фильтры горизонтально
display(widgets.HTML("<h3>Фильтры:</h3>"))
filter_box = widgets.HBox([
    widgets.VBox([widgets.HTML("<b>Образование</b>"), education_filter]),
    widgets.VBox([widgets.HTML("<b>Семейное положение</b>"), marital_filter]),
    widgets.VBox([widgets.HTML("<b>Наличие детей</b>"), children_filter]),
    widgets.VBox([widgets.HTML("<b> </b>"), update_button])
])
display(filter_box)

# Вывод для дашборда
display(output)

# Первоначальная отрисовка
update_dashboard(None)